In this notebook, we'll walk you through how to generate text using a character RNN model. Here are the topics we'll cover:
- Imports the required libraries
- Downloads the Shakespeare dataset
- Preprocesses the text data
- Defines a model architecture
- Compiles the model
- Trains the model
- Generates text using the trained model

In [ ]:
import tensorflow as tf

In [ ]:
path = "https://raw.githubusercontent.com/TirendazAcademy/Deep-Learning-with-TensorFlow/main/Data/tinyshakespeare.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", path)
with open(filepath) as f:
    text = f.read()

In [ ]:
print(text[:100])

In [ ]:
"".join(sorted(set(text.lower())))

In [ ]:
len("".join(sorted(set(text.lower()))))

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(
    split="character",standardize="lower")

In [ ]:
text_vec_layer.adapt([text])

In [ ]:
text_vec_layer([text]).shape

In [ ]:
encoded = text_vec_layer([text])[0]
encoded

In [ ]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size()-2 
n_tokens

In [ ]:
dataset_size = len(encoded)
dataset_size

In [ ]:
def to_dataset(sequence,length,shuffle=False,seed=None,batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1,drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(
        lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [ ]:
list(to_dataset(text_vec_layer(["I like"])[0], length=5))

In [ ]:
length = 100
tf.random.set_seed(42)

train_set = to_dataset(
    encoded[:1_000_000], length=length, shuffle=True,seed=42)

valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)

test_set = to_dataset(encoded[1_060_000:], length=length)

# Building and Training the Char-RNN Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128,return_sequences=True),
    tf.keras.layers.Dense(n_tokens,activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="nadam", metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint("my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(
    train_set, validation_data=valid_set, epochs=3,callbacks=[model_ckpt])

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

# Prediction

In [ ]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

# Generating Fake Shakespearean Text

In [ ]:
log_probas = tf.math.log([[0.6, 0.3, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=10)

In [ ]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)
print(extend_text("I like", temperature=0.01))

In [ ]:
print(extend_text("I like", temperature=1))